In [1]:
import h2o4gpu as sklearn
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import datasets

In [2]:
iris = datasets.load_iris()

In [3]:
df = pd.DataFrame(data = iris.data,columns = iris.feature_names)

In [4]:
df["target"] = iris.target
l = []
[l.append("_".join(i[:-5].split())) for i in df.columns[:-1]]
df.columns = l + ["target"]

In [5]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
import tensorflow as tf
tf_features = []
[tf_features.append(tf.feature_column.numeric_column(i)) for i in df.columns[:-1]]

tf_features

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [7]:
classifier = tf.estimator.DNNClassifier(hidden_units = [10,20,10], n_classes = 3, feature_columns = tf_features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp4xqyuv83', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f74e9c829e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
from sklearn.model_selection import train_test_split

X = df.drop("target",axis = 1)
y = df["target"]

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.7)

/home/saarth/anaconda3/envs/h2o4gpuenv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
input_fn = tf.estimator.inputs.pandas_input_fn(X_train,y_train,batch_size = 10,shuffle = True)

In [10]:
classifier.train(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp4xqyuv83/model.ckpt.
INFO:tensorflow:loss = 27.46482, step = 1
INFO:tensorflow:Saving checkpoints for 11 into /tmp/tmp4xqyuv83/model.ckpt.
INFO:tensorflow:Loss for final step: 2.6873417.


In [11]:
output_fn = tf.estimator.inputs.pandas_input_fn(X_test, batch_size = len(X_test),shuffle = False)

In [12]:
result = classifier.predict(output_fn)

result = list(result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4xqyuv83/model.ckpt-11
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [13]:
ypred = []

[ypred.append(i["class_ids"][0]) for i in result]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [14]:
from sklearn.metrics import confusion_matrix

confusion_matrix(ypred,y_test)

array([[18,  0,  0],
       [ 0,  0,  0],
       [ 0, 16, 11]])